In [1]:
import pandas as pd
import numpy as np

import tensorflow as tf

In [2]:
rat_df = pd.read_csv('./ratings_small.csv')

In [3]:
rat_df.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [4]:
rat_df['userId_2'] = rat_df['userId'] - 1
rat_df['movieId_2'] = rat_df['movieId'] - 1

In [5]:
num_movies = len(np.unique(rat_df['movieId_2']).tolist())
num_users = len(np.unique(rat_df['userId_2']).tolist())
num_features = 100

In [6]:
R = np.zeros((num_movies, num_users))
Y = np.zeros((num_movies, num_users))

movie_idx = sorted(np.unique(rat_df['movieId_2']))
uni_mov_idx = list(range(np.unique(rat_df['movieId_2']).shape[0]))

movie_idx_dict = dict(zip(movie_idx, uni_mov_idx))
rat_df['movieId_2'] = rat_df['movieId_2'].map(movie_idx_dict)

for i in range(num_users):
    movies_idx = rat_df[rat_df['userId_2'] == i].loc[:, 'movieId_2']
    rates_idx = rat_df[rat_df['userId_2'] == i].loc[:, 'rating']
    R[movies_idx, i] = 1
    Y[movies_idx, i] = rates_idx

Ymean = (np.sum(Y * R, axis=1)/(np.sum(R, axis=1)+1e-12)).reshape(-1,1)
Ynorm = Y - np.multiply(Ymean, R)

In [7]:
def criterion(w, x, b, lambda_, Y, R):
    """
    Returns the cost for the content-based filtering
    Vectorized for speed. Uses tensorflow operations to be compatible with custom training loop.
    
    
    Args:
      x (ndarray (num_movies, num_features)): matrix of item features
      w (ndarray (num_users,  num_features)): matrix of user parameters
      b (ndarray (1, num_users)             : vector of user parameters
      Y (ndarray (num_movies, num_users)    : matrix of user ratings of movies
      R (ndarray (num_movies, num_users)    : matrix, where R(i, j) = 1 if the i-th movies was rated by the j-th user
      lambda_ (float)                       : regularization parameter
      
    Returns:
      J (float) : Cost
    """
    J = (tf.linalg.matmul(x, tf.transpose(w)) + b - Y) * R
    J = 0.5 * tf.reduce_sum(J**2) + (lambda_/2) * (tf.reduce_sum(x**2) + tf.reduce_sum(w**2))
    return J

In [9]:
W = tf.Variable(tf.random.normal((num_users, num_features),dtype=tf.float64),  name='W')
X = tf.Variable(tf.random.normal((num_movies, num_features),dtype=tf.float64),  name='X')
b = tf.Variable(tf.random.normal((1, num_users),dtype=tf.float64),  name='b')

opt = tf.keras.optimizers.Adam(learning_rate=1e-1)

iterations = 400
lambda_ = 1

for iters in range(iterations):
    with tf.GradientTape() as tape:
        cost = criterion(W, X, b, lambda_, Ynorm, R)
        grads = tape.gradient(cost, [W, X, b])
        opt.apply_gradients(zip(grads, [W, X, b]))
        
        if (iters + 1) % 50 == 0:
            print(f"Training loss at iteration {iters + 1}: {cost:0.1f}")

Training loss at iteration 50: 76374.6
Training loss at iteration 100: 20307.4
Training loss at iteration 150: 9545.6
Training loss at iteration 200: 6447.4
Training loss at iteration 250: 5355.6
Training loss at iteration 300: 4896.6
Training loss at iteration 350: 4668.8
Training loss at iteration 400: 4538.8
